<a href="https://colab.research.google.com/github/richardachen/PS2HouseDataExercise/blob/master/PS2HousesPredictionPrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model

# url for the training data csv file
url = 'https://raw.githubusercontent.com/richardachen/PS2HouseDataExercise/master/train.csv'

houses = pd.read_csv(url)

In [ ]:
# https://drive.google.com/file/d/1byVJh3DNqpUvT28Euu1sG1Hc2bSKg9kG/view

#select only numerical data types for use 
numeric = houses.select_dtypes(include=[np.number])

# drop columns containing NaN values
numeric = numeric.dropna(axis = 1)

In [ ]:
# https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows

# shuffle dataframe rows, in case there was some grouping of rows in the original dataframe
numeric = numeric.sample(frac=1).reset_index(drop=True)

# use 900 rows for training, 100 for validation
train = numeric[0:900]
corr = train.corr()
validate = numeric[900:1000]

max_score = 0
preferred_depth = 0

# optimize for the best number of parameters to take in
for i in range(2,len(corr)):
  cols = corr['SalePrice'].sort_values(ascending = False)[0:i].index

  # get a dataframe with only the top i correlated columns
  train_params = train[cols]

  Y = train_params['SalePrice']
  X = train_params.drop(['SalePrice'], axis = 1)

  # fit the linear model with the new filtered dataframe
  lr = linear_model.LinearRegression()
  model = lr.fit(X,Y)

  # predict on the validation dataset
  valid_params = validate[cols]
  W = valid_params['SalePrice']
  Z = valid_params.drop(['SalePrice'], axis = 1)

  if model.score(Z,W) > max_score:
    preferred_depth = i
    max_score = model.score(Z,W)

print(preferred_depth)
print(max_score)


34
0.7935801776210235


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv

# now that we know the best depth to use, we can implement the best performing model
cols = corr['SalePrice'].sort_values(ascending = False)[0:preferred_depth].index
best_params = train[cols]

Y = best_params['SalePrice']
X = best_params.drop(['SalePrice'], axis = 1)
lr = linear_model.LinearRegression()
model = lr.fit(X,Y)

# Load in test dataset
test_url = 'https://raw.githubusercontent.com/richardachen/PS2HouseDataExercise/master/corrected_test.csv'

test_houses = pd.read_csv(test_url)

test_cols = cols.drop('SalePrice')
test_cols

# Apply predictive model to our test dataset
test_best_params = test_houses[test_cols]
test_best_params
predictions = model.predict(test_best_params)

# https://www.pythontutorial.net/python-basics/python-write-csv-file/
# Write predictions into a new csv file
f = open('/content/drive/MyDrive/Spring 2022/1.00/test_house_predict.csv', 'w')
writer = csv.writer(f)
writer.writerow(['Id', 'SalePrice'])
for i in range(len(predictions)):
  writer.writerow([test_best_params.iloc[i]['Id'],predictions[i]])